In [1]:
"""
main.ipynb

Main file for recreating lottery ticket experiments done in randomly initialized dense neural networks.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
%load_ext autoreload
%autoreload 2

import functools
import numpy as np
import os
import tensorflow as tf

from src.harness import constants as C
# from src.harness.dataset import download_data, load_and_process_mnist
from src.harness.experiment import ExperimentSummary, ExperimentData, run_experiments
# from src.harness.model import create_model, LeNet300, load_model
# from src.harness.pruning import prune_by_percent
# from src.harness.training import train
# from src.lottery_ticket.foundations import paths


ImportError: cannot import name 'run_experiments' from 'src.harness.experiment' (/Users/jordan/Projects/lottery-tickets/src/harness/experiment.py)

In [ ]:
summary: ExperimentSummary = run_experiments(num_models=C.TEST_NUM_MODELS,
                                             training_iterations=100,
                                             pruning_percents=C.TEST_PRUNING_PERCENTS,
                                             pruning_steps=C.TEST_PRUNING_STEPS)

Pruning Step 0
Iteration 1/100, Loss: 2.3527424335479736
Iteration 2/100, Loss: 2.2643282413482666
Iteration 3/100, Loss: 2.1938443183898926
Iteration 4/100, Loss: 2.129702568054199
Iteration 5/100, Loss: 2.067338705062866
Iteration 6/100, Loss: 2.0048041343688965
Iteration 7/100, Loss: 1.9412062168121338
Iteration 8/100, Loss: 1.876131534576416
Iteration 9/100, Loss: 1.8094804286956787
Iteration 10/100, Loss: 1.7416300773620605
Iteration 11/100, Loss: 1.673186182975769
Iteration 12/100, Loss: 1.6049703359603882
Iteration 13/100, Loss: 1.537709355354309
Iteration 14/100, Loss: 1.4719213247299194
Iteration 15/100, Loss: 1.4081066846847534
Iteration 16/100, Loss: 1.3467572927474976
Iteration 17/100, Loss: 1.2881991863250732
Iteration 18/100, Loss: 1.2326465845108032
Iteration 19/100, Loss: 1.1802276372909546
Iteration 20/100, Loss: 1.131006121635437
Iteration 21/100, Loss: 1.0849409103393555
Iteration 22/100, Loss: 1.0419408082962036
Iteration 23/100, Loss: 1.0019105672836304
Iteration 2

In [ ]:
print(summary)